In [1]:
import re
import pandas as pd
from pprint import pprint

# Perapian lebih lanjut

In [2]:
katalog = pd.read_csv('katalog_cleaned.tsv', sep='\t').fillna('')

In [3]:
kat_length = katalog.count()['title']
kat_length

53365

## Kolom `contributor`

Alasan:
* Penulisan gelar yang tidak konsisten mengganggu dalam menganalisis/membuat model
* Begitu pula dengan teks non nama, seperti "Pembimbing Tugas Akhir", atau "Advisor :"

Tindakan:
* hapus penulisan gelar dan teks non-nama, sebisanya.

Cara:
1. rapikan dan hapus kerusakan-kerusakan umum
2. hapus penulisan gelar
3. perbaiki kesalahan format nama, dan kerusakan signifikan yang terlewat


### Kerusakan umum

In [4]:
na_list = {'-':'', 'null':'', 'tidak ada':'', '#CONTRIBUTOR#':''}
katalog['contributor'].replace(na_list, inplace=True)

In [5]:
def clean1(val):
    # karakter tilda (~) sebagai pemisah antar nama
    
    # ===================================
    # tidak ada nama pada teks yang mengandung mahmudin@
    if 'mahmudin@' in val: return ''
    # hapus alamat email
    val = re.sub('\w.*@.*\.(id|com)', '', val)
    # hapus karakter-karakter berikut
    val = re.sub('&#\d+\;', '', val)
    val = re.sub('(Ã(‚|ƒ)Â|ƒÃ‚)','', val)
    val = re.sub('(<br|br>)', '', val)
    
    # ===================================
    # nama orang yang menge-scan dokumen
    val = re.sub('scan(ner){,1}[ :]*.*', ' ', val, flags=re.I)
    val = re.sub('(Ena Sukmana|editor| oleh |unggah pertama pada)', ' ', val, flags=re.I)
    # teks non-nama seperti "Dosen Pembimbung I:", "Advisor 1 :", dan sejenisnya
    # anggap teks ini sebagai pemisah antar nama
    val = re.sub('(dosen){,1}( ){,3}(pe.{4,8}ng|advisor|akademik)( )*(utama|tesis|pertama|kedua|proyek akhir|tugas akhir|I{,3}|\d)( )*(:){,1}', ' ~ ', val, flags=re.I)
    val = re.sub('(|ko-|co-)(supervisor|promotor|penulis|pembina|author(s){,1})[ I\d]*?:', ' ~ ', val, flags=re.I)
    val = re.sub('Koordinator (Kelompok|Tugas Akhir|TA Desain Produk)', ' ~ ', val, flags=re.I)
    val = re.sub('Ketua( Program Studi){,1}',' ~ ', val, flags=re.I)

    # ===================================
    # hilangkan gelar yang sering dijumpai + tidak konsisten
    # anggap teks ini sebagai pemisah antar nama
    val = re.sub('Dr(a|s){,1}.*?(nat|pol|tech(n){,1})( |\.)', ' ~ ', val, flags=re.I)
    val = re.sub('Dr(a|s){,1}.{,3}(ing|eng|ir)( |\.)', ' ~ ', val, flags=re.I)
    val = re.sub('Dr(a|s){,1}\.', ' ~ ', val, flags=re.I)
    val = re.sub('Ph( |\.|\. )D( |\.)', ' ~ ', val, flags=re.I)
    val = re.sub('PhD( |\.)', ' ~ ', val, flags=re.I)
    val = re.sub('((P|p)rof|Ir)\.', ' ~ ', val)
    val = re.sub('M( |\.|\. )B( |\.|\. )A( |\.|\. )', ' ~ ', val)
    val = re.sub('MBA', ' ~ ', val)

    # ===================================
    # anggap teks ini sebagai pemisah antar nama
    val = re.sub('[\,\(\)\:;•‡†§><&#]', ' ~ ', val)
    val = re.sub('\d+', ' ~ ', val)
    # rapikan spasi berlebih
    val = re.sub('\s+', ' ', val)
    # sederhanakan pemisah antar nama, (~ ~ ~) menjadi (~) saja
    val = re.sub('~( ~)+', '~', val)

    return val.strip()

In [6]:
katalog['contributor'] = katalog['contributor'].apply(clean1)
tmp = katalog['contributor'].value_counts().to_dict()

# persentase teks unik
print(100 * len(tmp)/kat_length)

42.555982385458634


### Menghapus gelar

In [7]:
# daftar gelar, lupa ngambil dari mana, sebagian dari feedback-loop output kodingan
gelar = ['Prof', 'M Des', 'M Sc', 'M Sc.', 'M Sn.', 'M-Eng', 'M. D.', 'M. Ds', 'M. E', 'M. E.', 'M. Ec', 'M. Ed', 'M. H', 'M. IP', 'M. LA', 'M. M.', 'M. Mo', 'M. S.', 'M. SA', 'M. Sc', 'M. Si', 'M. Sn', 'M. T', 'M. T.', 'M. UP', 'M. s', 'M. s.', 'M. sc', 'M. si', 'Sp.B', 'Sp.F.', 'Sp.FK', 'Sp.JP', 'Sp.KJ', 'Sp.M.', 'Sp.P', 'Sp.P.', 'Sp.PK', 'Sp.S', 'Sp.S.', 'SpFK', 'SpJP', 'SpJp', 'SpKFR', 'SpKJ', 'SpPD', 'SpPD.', 'SpPK', 'SpPK.', 'MSc..', 'MSc.Apt.', 'MSc.CE.', 'MSc.EE', 'MScE.', 'MScEE.', 'MScI', 'MSi.', 'MSn.', 'M.S.E.', 'Apt.', 'M.Eng.', 'MAUD', 'MUDD', 'ST. MT.' 'MS', 'M.Sc.', 'DEA', 'A.Ma.Pd', 'S.P', 'M.Kor.', 'M.H.', 'S.Kel.', 'B.Sc', 'M.Hum', 'M.P.H', 'M.Ked.Trop.', 'M.P.I.', 'M.E.I.', 'Dra', 'S.Si', 'M.Kes.', 'B.Eng', 'S.Sos', 'Mbus', 'M.Han.', 'M.AP.', 'M.T.', 'M.I.Kom.', 'S.Sn', 'S.Kep.', 'M.T.A', 'M.Ed', 'S.Sn.', 'S.T', 'M.Fhil.', 'S.Ars', 'B.E', 'M.A.Ked.', 'S.Pd.SD', 'MA.', 'M.Arch', 'L.L.B', 'M.Mgt', 'S.Farm', 'S.Pd.SD.', 'M.I.Pol.', 'S.IKom', 'S.S.T', 'MSEE.', 'S.P.', 'M.A.Pd.', 'M.Lib', 'M.Sc', 'S.H.I', 'S.SI.', 'M.Farm.Klin.', 'M.Hut.', 'S.Kom.', 'S.I.P.', 'M.H.Kes.', 'M.P.Kim.', 'A.P.Kom.', 'A.Md.Pd.', 'MMSI', 'M.K.K.', 'S.Kom', 'A.Md.K.G.', 'M.Li.', 'MMSI.', 'S.Gz.', 'MKKK', 'D.P.H', 'S.E', 'A.Md.Bid.', 'S.H.', 'S.Hum', 'M.A.Hum.', 'S.H', 'M.Pd', 'M.Ds.', 'M.F', 'M.P.Kim', 'M.P.H.', 'S.Pd.I.', 'B.Ag', 'M.Sos.I.', 'MComm', 'S.Hut.', 'M.Min', 'S.Ked', 'S.Fhil', 'A.Ma.Pust.', 'S.Mn', 'M.P', 'MSIE', 'Dr.H.C', 'S.Th.I.', 'M.M.', 'M.Ak.', 'M.Comp.Sc', 'S.Pt.', 'S.Pd', 'A.Md.Far.', 'D.M.D', 'M.Hum.', 'MSA', 'A.Md.Par', 'Th.M', 'SKG.', 'S.Psi.', 'M.A.R.S.', 'S.T.P.', 'M.B.A', 'S.Pd.I', 'S.Pi.', 'S.T.', 'M.Kesja', 'S.Sos.I', 'S.AP.', 'A.P', 'M.Eng', 'M.Mar', 'S.Pt', 'S.S.', 'B.Bus', 'A.Ma.', 'M.App.Sc', 'Th.D', 'M.R.E', 'S.E.', 'M.Kes', 'S.Ag.', 'M.Fil.I.', 'M.F.A', 'S.Kel', 'S.Kep', 'Dra.', 'M.Litt', 'M.Si', 'M.Psi.', 'Drs', 'M.Stat.', 'M.Nurs', 'M.Sn.', 'S.Fhil.', 'S.K.M', 'SEi', 'M.Tr.', 'S.I.Kom.', 'MSIE.', 'S.Kes', 'B.D', 'M.Kom', 'B.Com', 'M.Mus', 'B.Arch', 'MKK', 'A.Md.Kom.', 'M.Pd.I.', 'S.Psi', 'M.Si.', 'S.Pi', 'M.Ag', 'M. Kn', 'MMR', 'S.E.I.', 'M.Agri.', 'Ed.D', 'B.Litt', 'M.Farm.', 'S.In', 'D.Sc', 'A.Md.Par.', 'S.Ag', 'S.Pd.', 'A.P.Par.', 'M.Ars.', 'Ir.', 'S.Apt', 'S.Th.I', 'S.Agr.', 'S.Ked.', 'M.MPd.', 'S.K.M.', 'A.Md.Kes.', 'M.Biomed.', 'M.Epid.', 'Pharm.D', 'J.S.D', 'BIE', 'B.A', 'M.S.I.', 'S.si', 'M.Tr.Hanla.', 'M.Ked', 'Bc,Kn', 'S.IP.', 'M.P.Mat.', 'S.Kar', 'Mcom', 'M.E.Sy.', 'M.A.', 'S.S', 'A.Md.Bid', 'M.Phil', 'S. AB.', 'A.Ma.Pd.', 'M.S', 'S.I.P', 'MARS', 'A.Md.Per.', 'MSA.', 'S.Th.K', 'Dr.', 'M.Sn', 'S.Ners', 'S.Sy.', 'A.Md.Keb.', 'S.Hut', 'DBA', 'A.Md.Kes', 'M.Th', 'M.M', 'S.KG', 'A.Md.Per', 'M.T', 'Ir', 'S.H.I.', 'M.H.I.', 'BBA', 'D.Econ', 'Dr', 'S.Sos.', 'M.Div.', 'A.Md.Ak.', 'L.L.M', 'M.AB.', 'M.Eng.Sc', 'A.P.', 'S.Ds', 'B.M', 'D.L.S', 'M.S.M.', 'S.STP.', 'M.A', 'M.Kn.', 'M.Si.Han', 'Drs.', 'S.Si.', 'M.TI.', 'S.Gz', 'A.Md', 'S.TP', 'A.Md.', 'S.KH', 'M.Keb.', 'M.Kom.', 'S. Farm.', 'M.Kesos.', 'M.Econ.', 'S.Fil.I', 'A.Ma', 'M.P.', 'M.Ag.', 'D.Comm', 'A', 'M.Cs.', 'Dipl. Ing', 'M.Pd.', 'Ph.D', 'D.Eng', 'M.Econ']
gelar = set(gelar)

In [8]:
def clean2(val):
    # hapus nama gelar
    val = ' '.join([w for w in val.split() if (w not in gelar)])
    # anggap `dan` dan `and` sebagai pemisah nama
    # agar tidak sengaja memroses nama, ikutkan spasi
    val = re.sub('( dan ~ | ~ dan )', '~', val, flags=re.I)
    val = re.sub('( and ~ | ~ and |\?and)', '~', val, flags=re.I)
    # sederhanakan pemisah antar nama
    val = re.sub('~( ~)+', '~', val)
    if val and val[0]=='~': val=val[2:]
    if val and val[-1]=='~': val=val[:-2]

    return val

In [9]:
katalog['contributor'] = katalog['contributor'].apply(clean2)
tmp = katalog['contributor'].value_counts().to_dict()

# persentase teks unik
print(100 * len(tmp)/kat_length)

35.182235547643586


### Merapikan gaya penulisan

In [10]:
def clean3(val):
    # ekstrak nama yang didapat hasil cleaning sebelumnya
    tmp = [w.strip() for w in val.split('~')]
    tmp1 = []
    for w in tmp:
        # hapus tanda titik jika muncul sebagai karakter pertama
        if w and w[0]=='.': w=w[1:]
        # jika ada huruf kapital diikuti tanda titik, kasih spasi
        # konstrain dibuat untuk menghindari beberapa gelar
        w = re.sub('([A-Z]\.)', '\\1 ', w)
        # jika ada nama yang diakhiri tanda titik, hilangkan tanda
        # titik. dilakukan dengan melihat apakah tanda titik
        # diawali dengan tiga karakter huruf kecil, contoh
        # "Brahmono." => "Brahmono"
        w = re.sub('([a-z]{3})\.', '\\1', w)
        # rapikan spasi ganda
        w = re.sub('( )+', ' ', w)
        w = w.strip()
        # hanya ikutkan jika di akhir proses jumlah karakter
        # non titik tidak kurang dari 3
        if len(w.replace('.',''))>=3: tmp1.append(w)

    # gabungkan semua nama kembali
    val = ' ~ '.join(tmp1)
    
    return val

In [11]:
katalog['contributor'] = katalog['contributor'].apply(clean3)
tmp = katalog['contributor'].value_counts().to_dict()

# persentase teks unik
print(100 * len(tmp)/kat_length)

31.338892532558795


In [12]:
katalog['contributor'].sample(10)

19371                                    Eka Djunarsjah
3806                                                   
31288         Prayatni Soewondo ~ Ahmad Soleh Setiyawan
37232                                                  
50089                                  Andhika Sahadewa
21070                     Hernawan Mahfudz ~ Moh. Farid
22793      Yazid Bindar ~ Made Tri Ari Penia Kresnowati
13201    Emmy Suparka ~ Niniek Rina Herdianita ~ M. Sc.
52164                            Sudarso Kaderi Wiryono
47476                                apt Ilma Nugrahani
Name: contributor, dtype: object

## Kolom `creator`

Alasan:
* Pada dasarnya sama dengan kolom `description`
* Ada cukup banyak irisan nama yang disimpan dalam kolom `creator` dan kolom `contributor`

Tindakan:
* lakukan perapian yang sama seperti kolom `description`
* hilangkan irisan nama antara kolom `creator` dan kolom `contributor`

In [13]:
katalog['creator'].loc[14797]

'TRI F ; Tim Pembimbing : Dr. Ir. Toto Hardianto; Prof. Dr. Ir. Aryadi Suwon, ANDIEK'

In [14]:
# CLEANING
clean_list = {
    'Central Library':'', 'Anonim':'', 'undefined':''
}
katalog['creator'].replace(clean_list, inplace=True)

In [15]:
def clean4(val):
    # ubah gaya penulisan nama menjadi
    # first name last name
    *last, first = val.split(',')
    val = first + ' ' + ','.join(last)
    return val

In [16]:
tmp = katalog['creator']

# lakukan cleaning, termasuk cleaning
# yang sama dengan kolom contributor
tmp = tmp.apply(clean4)
tmp = tmp.apply(clean1)
tmp = tmp.apply(clean2)
tmp = tmp.apply(clean3)

katalog['creator'] = tmp

In [17]:
def clean_creator(val):
    cr = set(val[0].split(' ~ '))
    co = set(val[1].split(' ~ '))
    
    if len(cr)>1: cr = cr-co
    return ' ~ '.join(cr)

def clean_contributor(val):
    cr = set(val[0].split(' ~ '))
    co = set(val[1].split(' ~ '))
    
    return ' ~ '.join(co-cr)

katalog['creator'] = katalog[['creator', 'contributor']].apply(clean_creator, axis=1)
katalog['contributor'] = katalog[['creator', 'contributor']].apply(clean_contributor, axis=1)

In [18]:
katalog[['creator', 'contributor']].sample(3)

,creator,contributor
25227,ALFIAN BAHAR,Djoko Santoso ~ Sri Widiyantoro
24901,ADI RAJA SURYA SIMAMORA -,Indra Djati Sidi
48318,Salsabila Tantri Ayu,Rachmawati


## Kolom `description`

Alasan:
* Menyatukan teks-yang-mensyaratkan-data-NA agar sedikit waras

Tindakan:
* Satukan datanya.

Cara:
* 300 karakter harusnya cukup untuk menulis NA.
* Gunakan fuzzy search untuk mengelompokkan NA.
* ganti daftar teks yang didapat dengan empty string.

In [19]:
from fuzzywuzzy import fuzz

In [20]:
desc = katalog['description']

# fokus ke teks dengan panjang kurang dari 300 karakter
tmp = desc[desc.str.len() < 300].to_list()
# fokus pada daftar teks yang unik
tmp = list(set(tmp))

In [21]:
%%time

group = []

def grouper():
    # menggabungkan beberapa teks yang "sama"
    # ke dalam satu kelompok
    text = tmp.pop()
    group.append([text])
    
    for other_text in tmp:
        # teks dianggap sama jika rasionya > 85
        if fuzz.ratio(text, other_text)>85:
            group[-1].append(other_text)
            tmp.remove(other_text)

# lakukan sampai semua teks di tmp
# berhasil dikelompokkan
while tmp: grouper()

CPU times: user 21.9 s, sys: 1.31 ms, total: 21.9 s
Wall time: 21.9 s


In [22]:
na_list = {}

for g in group:
    # fokus pada group dengan anggota lebih dari satu
    if len(g)==1: continue
    
    # semua teks-mensyaratkan NA punya panjang < 70
    for text in g:
        # masukkan ke daftar
        if len(text)<70: na_list[text]=''
    
print(len(na_list))

69


In [23]:
katalog['description'] = katalog['description'].replace('clean_list')

In [ ]:
katalog.to_csv('katalog_cleaned_v2.tsv', sep='\t', index=False)